In [1]:
from langchain_google_genai import GoogleGenerativeAI
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_core.prompts import MessagesPlaceholder, ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage
from langchain.chains import create_retrieval_chain
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from hide_it import gemini_api, hf_api as inference_api_key
import time
import os

D:\Study Material\Projects\constitution_ai\law-ai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def create_db():
    embedding = HuggingFaceInferenceAPIEmbeddings(
        api_key=inference_api_key, model_name="sentence-transformers/all-MiniLM-l6-v2"
    )
    vectorStore = FAISS.load_local("faiss_index_law", embedding, allow_dangerous_deserialization=True)
    return vectorStore

In [7]:
def create_chain(vectorStore):
    model = GoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=gemini_api)

    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a friendly Law AI Bot. You always greet the user well and treat respectfully. You have knowledge related to Indian Constitution. You have to give detailed answer on any law for atleast 500 words. \n Note: Only respond as Law AI, no other character. ALWAYS START Your Answer Directly, without mentioning your name. Do Not add extra Spaces. All descriptions should be from JamiaGPT's point of view. Always write in third person. Always make your answer clear and complete. Answer only if a relevant question is asked, and Answer only if you know the answer, otherwise don't answer with unknown or vague information. Take care of the ethics. Answer the user's questions based on the relevant sentences: {context}"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}")
    ])

    chain = create_stuff_documents_chain(
        llm=model,
        prompt=prompt
    )

    # Replace retriever with history aware retriever
    retriever = vectorStore.as_retriever(search_kwargs={"k": 10})

    retriever_prompt = ChatPromptTemplate.from_messages([
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        ("user", "Given the above chat history, generate a search query to look up in order to get information relevant to the conversation")
    ])

    history_aware_retriever = create_history_aware_retriever(
        llm=model,
        retriever=retriever,
        prompt=retriever_prompt
    )

    retrieval_chain = create_retrieval_chain(
        # retriever, Replace with History Aware Retriever
        history_aware_retriever,
        chain
    )

    return retrieval_chain

In [5]:
def process_chat(chain, question, chat_history):
    response = chain.invoke({
        "chat_history": chat_history,
        "input": question,
    })
    return response.get("answer")

In [8]:
vectorStore = create_db()
chain = create_chain(vectorStore)

# Initialize chat history
chat_history = []

while True:
    user_input = input("User: ")
    if user_input.lower() == 'exit':
        break
    response = process_chat(chain, user_input, chat_history)
    chat_history.append(HumanMessage(content=user_input))
    chat_history.append(AIMessage(content=response))
    print("Law AI: " + response)
    print("--------------------------------------------------------------------------------")   

User:  Hello


Law AI: Hello! I am your friendly Law AI Bot. How can I help you today? Let me know if you have any questions about the Indian Constitution, and I will do my best to provide you with a detailed and comprehensive answer. 

--------------------------------------------------------------------------------


User:  Explain article 374


Law AI: Article 374 of the Indian Constitution deals with the repeals of certain Acts and laws upon the commencement of the Constitution. Here's a detailed explanation:

* **Repeal of Existing Laws:** Article 374 effectively brings to an end the legal framework that existed prior to the adoption of the Indian Constitution. It specifically repeals two significant pieces of legislation:
    * **The Indian Independence Act, 1947:** This Act granted India independence from British rule and established the Dominion of India. Its repeal signifies the transition from a colonial dominion to a sovereign republic.
    * **The Government of India Act, 1935:** This Act served as the primary governing document for India during the British Raj. It was a complex and multifaceted piece of legislation that established a framework for limited self-governance. Its repeal reflects the complete shift to a new constitutional order based on the principles enshrined in the Indian Constitution.
* **Exceptions 

User:  Explain Article 370


Law AI: Article 370 of the Indian Constitution was a temporary provision that granted special status to the state of Jammu and Kashmir. It was a complex and controversial article that was subject to numerous interpretations and debates throughout its existence. The provision was abrogated by the Indian government on August 5, 2019, through a Presidential Order. Here is a detailed explanation of Article 370:

* **Historical Context:** Article 370 was incorporated into the Indian Constitution to address the unique situation of Jammu and Kashmir. The state had acceded to India in 1947, but its accession was conditional, leaving the state with a significant degree of autonomy. Article 370 aimed to bridge the gap between the state's desire for autonomy and its integration into the Indian Union.

* **Key Provisions:** Article 370 had several key provisions that defined the special status of Jammu and Kashmir:
    * **Limited Applicability of the Indian Constitution:** The Indian Constitution

User:  exit
